In [1]:
import numpy as np
import torch
import torch.nn as nn
from torch.utils.data import DataLoader
from torchvision import transforms, models
from torchvision.models import ResNet50_Weights

from create_dataset import ClimbingHoldDataset, ClimbingHoldDatasetPred

In [2]:
# Choose device
if torch.cuda.is_available():
    device = torch.device("cuda")
elif hasattr(torch.backends, "mps") and torch.backends.mps.is_available():
    device = torch.device("mps")  # Apple Silicon
else:
    device = torch.device("cpu")

print("Using device:", device)

IMG_SIZE = 224

# Evaluation transform (no heavy augmentations)
eval_transform = transforms.Compose([
    transforms.Resize((IMG_SIZE, IMG_SIZE)),
    transforms.ToTensor(),
    transforms.Normalize(
        mean=[0.485, 0.456, 0.406],
        std=[0.229, 0.224, 0.225],
    ),
])


Using device: mps


In [3]:
annotations_dir = "hold_preds"  # <-- change this
images_dir = "data/images"            # <-- change this

dataset = ClimbingHoldDatasetPred(
    annotations_dir=annotations_dir,
    images_dir=images_dir,
    output_size=(IMG_SIZE, IMG_SIZE),
)

dataset.transform = eval_transform  # use eval transform

print("Total samples in NEW dataset:", len(dataset))


Total samples in NEW dataset: 4541


In [4]:
class TwoHeadResNet50(nn.Module):
    def __init__(self, num_types=7, num_orientations=5, pretrained=False, dropout=0.3):
        super().__init__()
        weights = ResNet50_Weights.IMAGENET1K_V1 if pretrained else None
        backbone = models.resnet50(weights=weights)

        in_features = backbone.fc.in_features
        backbone.fc = nn.Identity()
        self.backbone = backbone

        self.type_head = nn.Sequential(
            nn.Linear(in_features, 512),
            nn.ReLU(),
            nn.Dropout(dropout),
            nn.Linear(512, num_types),
        )

        self.orientation_head = nn.Sequential(
            nn.Linear(in_features, 256),
            nn.ReLU(),
            nn.Dropout(dropout),
            nn.Linear(256, num_orientations),
        )

    def forward(self, x):
        feats = self.backbone(x)
        type_logits = self.type_head(feats)
        orientation_logits = self.orientation_head(feats)
        return type_logits, orientation_logits


In [5]:
num_types = 7
num_orients = 5

model = TwoHeadResNet50(
    num_types=num_types,
    num_orientations=num_orients,
    pretrained=False,  # weights come from checkpoint
    dropout=0.3,
).to(device)

state_dict = torch.load("best_hold_model.pth", map_location=device)
model.load_state_dict(state_dict)
model.to(device)
model.eval()

print("Loaded trained model from best_hold_model.pth")


Loaded trained model from best_hold_model.pth


In [6]:
from torch.utils.data import DataLoader

loader = DataLoader(dataset, batch_size=64, shuffle=False)

all_pred_types = []
all_pred_orients = []

with torch.no_grad():
    for batch in loader:
        imgs = batch["image"].to(device)  # from __getitem__
        out_type, out_orient = model(imgs)

        pred_type_batch = out_type.argmax(1).cpu().numpy()
        pred_orient_batch = out_orient.argmax(1).cpu().numpy()

        all_pred_types.append(pred_type_batch)
        all_pred_orients.append(pred_orient_batch)

pred_type_idx = np.concatenate(all_pred_types)   # shape (N,)
pred_orient_idx = np.concatenate(all_pred_orients)

print("Predicted types shape   :", pred_type_idx.shape)
print("Predicted orients shape :", pred_orient_idx.shape)


Predicted types shape   : (4541,)
Predicted orients shape : (4541,)


In [7]:
np.save("pred_type_idx.npy", pred_type_idx)
np.save("pred_orient_idx.npy", pred_orient_idx)

print("Saved pred_type_idx.npy and pred_orient_idx.npy")

Saved pred_type_idx.npy and pred_orient_idx.npy
